## 1. Importações e Setup

In [0]:
import delta
import sys

sys.path.insert(0, "../lib/")

import utils
import ingestors

In [0]:
catalog = "bronze"
schemaname = "upcell"

#tablename = "clientes"
#id_field = "idCliente"
#timefield = "DtAtualizacao"

## Access widget values outside of any asynchronous context
tablename = dbutils.widgets.get("tablename")
id_field = dbutils.widgets.get("id_field")
timefield = dbutils.widgets.get("timefield")

cdc_path = f"/Volumes/raw/{schemaname}/cdc/{tablename}/"
full_load_path = f"/Volumes/raw/{schemaname}/full-load/{tablename}/"
ckeckpoint_location = f"/Volumes/raw/{schemaname}/cdc/{tablename}_checkpoint/"

In [0]:
if not utils.table_exists(spark, catalog, schemaname, tablename):
    
    print("tabela nao existe, criando...")
    
    dbutils.fs.rm(ckeckpoint_location, True)

    ingest_full_load = ingestors.Ingestor(spark=spark,
                                          catalog=catalog, 
                                          schemaname=schemaname, 
                                          tablename=tablename, 
                                          data_format="parquet")
    ingest_full_load.execute(full_load_path)
    print("tabela criada com sucesso")

else:
    print("tabela ja existe, ignorando full-load")

In [0]:
ingest_cdc = ingestors.IngestorCDC(spark=spark,
                                   catalog=catalog, 
                                   schemaname=schemaname, 
                                   tablename=tablename, 
                                   data_format="parquet", 
                                   id_field=id_field,
                                   timefield=timefield)

stream = ingest_cdc.execute(cdc_path)